In [1]:
import time
import json
import importlib
import scipy.optimize as opt
import dlm
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
import warnings
from sklearn.metrics import classification_report,f1_score, precision_score, recall_score, accuracy_score
warnings.filterwarnings("ignore")

In [5]:
def preprocess_by_nomr(df):
    
    Y_train = []

    
    labels_Y = []
  
    for intensity in range(3):

        Y_data = df[intensity]


        Y_data["3D_acc_norm"] = np.sqrt(Y_data[4]**2 + Y_data[5]**2 + Y_data[6]**2)
        Y_data["3D_gyr_norm"] = np.sqrt(Y_data[10]**2 + Y_data[11]**2 + Y_data[12]**2)

        Y_train.append(Y_data[["3D_acc_norm","3D_gyr_norm"]].iloc[:])

    
        labels_Y.append([intensity]*len(Y_train[intensity]))
    
    
        
    labels_Y = np.concatenate(labels_Y)
    
    return Y_train, labels_Y

In [3]:
low_intense_acts = [1,2,3,4,17]

middle_intense_acts = [13,16,7] 

high_intense_acts = [5,6,24,12]

def report_results(subjectID):

    df_train = pd.read_csv(f"subject10{subjectID}.dat",sep=" ",header=None)
    df_train = df_train.interpolate()
  



    df_low_intense_acts = df_train[df_train[1].isin(low_intense_acts)]
    df_middle_intense_acts = df_train[df_train[1].isin(middle_intense_acts)]
    df_high_intense_acts = df_train[df_train[1].isin(high_intense_acts)]

    df_train = [df_low_intense_acts, df_middle_intense_acts, df_high_intense_acts]
    
    Y_train,labels_Y_train = preprocess_by_nomr(df_train)

    bdlm = dlm.ensemble_BDLM(n_acts = 3)
    
    bdlm.fit(Y_train)

    
    user_stats = {}

    user_stats["subjectID"] = f"subject{subjectID}"
    
    user_stats["labels_Y_train"] = labels_Y_train
    
    user_stats["Y_train"] = Y_train
    
    user_stats["bdlm"] = bdlm 
 
    
    return user_stats


In [6]:
subjects_report = []

for i in range(1,9):
    
    stats = report_results(subjectID = i)
    subjects_report.append(stats)
    
    print(f"Finished Training subject {i}")
    

Finished Training subject 1
Finished Training subject 2
Finished Training subject 3
Finished Training subject 4
Finished Training subject 5
Finished Training subject 6
Finished Training subject 7
Finished Training subject 8


In [7]:
macro_precision = []
macro_recall = []
accuracy = []

for j in range(len(subjects_report)):

    Y_subject = subjects_report[j]["Y_train"]
    y_test = subjects_report[j]["labels_Y_train"]
    subjectID = subjects_report[j]["subjectID"]

    voting_p = []
    
    for i in range(len(subjects_report)):

        if subjects_report[i]["subjectID"] == subjectID:

            continue

        else:
            probs = []
            for activity in Y_subject:

                _, p = subjects_report[i]["bdlm"].predict(activity, return_probs=True,restart=True)
                del _
                probs.append(p)

        voting_p.append(probs)

    p = np.zeros(np.concatenate(voting_p[0]).shape)
    
    for probability in voting_p:
        p += np.concatenate(probability)

    N = len(voting_p)
    p = p/N

    y_hat = np.argmax(p, axis=1)

    precision = precision_score(y_test,y_hat,average = "macro")
    recall = recall_score(y_test,y_hat,average = "macro")
    accuracy_ = accuracy_score(y_test,y_hat)

    macro_precision.append(precision)
    macro_recall.append(recall)
    accuracy.append(accuracy_)

    print(f"Subject {j}")

Subject 0
Subject 1
Subject 2
Subject 3
Subject 4
Subject 5
Subject 6
Subject 7


In [8]:
macro_precision = np.array(macro_precision)    
macro_recall = np.array(macro_recall)    
accuracy = np.array(accuracy)  

model_report = {}

model_report["PRECISION"] = [round(macro_precision.mean(), 4),round(macro_precision.std(), 4)]
model_report["RECALL"] = [round(macro_recall.mean(), 4),round(macro_recall.std(), 4)]
model_report["ACCURACY"] = [round(accuracy.mean(), 4),round(accuracy.std(), 4)]
model_report

{'PRECISION': [0.526, 0.059],
 'RECALL': [0.4754, 0.0487],
 'ACCURACY': [0.4952, 0.0319]}

In [9]:
df_reports = pd.DataFrame({
    'Subject': [f'subject{i}' for i in range(1, 9)],
    'Accuracy': accuracy,
    'Macro Precision': macro_precision,
    'Macro Recall': macro_recall
})

df_reports.to_csv("cross_user_BDLM.csv",index=False)

In [12]:
macro_precision

array([0.53945605, 0.59173627, 0.58025843, 0.61454315, 0.47794626,
       0.46976915, 0.45871484, 0.47523853])